# Install these libraries first if you haven't already [pandas, pyodbc, and sqlalchemy]

In [1]:
# !pip install pandas pyodbc sqlalchemy

In [2]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


# # Reset warnings to default behavior
# warnings.resetwarnings()


# Connect to MS SQL Server and Create a Database

In [3]:
import pyodbc

# Connection parameters
server = 'DESKTOP-89C0DB2'
database = 'master'  # or any other existing database where you want to check/create a new database
username = ''
password = ''
driver = '{SQL Server}'

# Create a connection to the server
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str, autocommit=True)  # Set autocommit to True

# Create a cursor to execute SQL queries
cursor = conn.cursor()

# Define the new database name
new_db_name = 'SQLTestOne'

# Check if the database already exists
cursor.execute(f"SELECT database_id FROM sys.databases WHERE name = '{new_db_name}'")
existing_database = cursor.fetchone()

if existing_database:
    print(f"The database '{new_db_name}' already exists.")
else:
    # Create a new database
    cursor.execute(f'CREATE DATABASE {new_db_name}')
    print(f"The database '{new_db_name}' has been created.")

# No need to commit when autocommit is set to True

# Close the cursor and connection
cursor.close()
conn.close()


The database 'SQLTestOne' has been created.


# Connect to the Newly Created Database and Create a Table

In [4]:
# Connect to the newly created database
new_db_conn_str = f'DRIVER={driver};SERVER={server};DATABASE={new_db_name};UID={username};PWD={password}'

try:
    new_db_conn = pyodbc.connect(new_db_conn_str)
    new_db_cursor = new_db_conn.cursor()

    # Define table name and columns
    table_name = 'YourTable'
    columns = [('ID', 'INT'), ('Name', 'VARCHAR(255)'), ('Age', 'INT')]

    # Create a table
    create_table_query = f'CREATE TABLE {table_name} ({", ".join([f"{col[0]} {col[1]}" for col in columns])})'
    new_db_cursor.execute(create_table_query)

    # Commit the transaction
    new_db_conn.commit()

    print("Table created successfully!")

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    # Close the cursor and connection in the finally block to ensure they are closed even if an exception occurs
    try:
        new_db_cursor.close()
        new_db_conn.close()
    except:
        pass


Table created successfully!


# Insert Data into the Table

In [5]:
# Connect to the database
try:
    new_db_conn = pyodbc.connect(new_db_conn_str)
    new_db_cursor = new_db_conn.cursor()

    # Define table name and data to insert
    table_name = 'YourTable'
    data_to_insert = [(1, 'John Doe', 25), (2, 'Jane Smith', 30), (3, 'Bob Johnson', 22)]

    # Insert data into the table
    insert_query = f"INSERT INTO {table_name} (ID, Name, Age) VALUES (?, ?, ?)"
    new_db_cursor.executemany(insert_query, data_to_insert)

    # Commit the transaction
    new_db_conn.commit()

    print("Data inserted successfully!")

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    # Close the cursor and connection in the finally block to ensure they are closed even if an exception occurs
    try:
        new_db_cursor.close()
        new_db_conn.close()
    except:
        pass


Data inserted successfully!


# CRUD Data and View as Pandas DataFrame

In [6]:
import pandas as pd

# Connect to the database
try:
    new_db_conn = pyodbc.connect(new_db_conn_str)
    new_db_cursor = new_db_conn.cursor()

    # CRUD operations
    # ... (perform your CRUD operations using SQL queries)

    # Read data into a Pandas DataFrame
    select_query = f"SELECT * FROM {table_name}"
    data_df = pd.read_sql(select_query, new_db_conn)

    # Display the DataFrame
    print("Data read successfully!")
    print("--------------------------------------------")
    print(data_df)

    

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    # Close the cursor and connection in the finally block to ensure they are closed even if an exception occurs
    try:
        new_db_cursor.close()
        new_db_conn.close()
    except:
        pass


Data read successfully!
--------------------------------------------
   ID         Name  Age
0   1     John Doe   25
1   2   Jane Smith   30
2   3  Bob Johnson   22


# SQL Problems Handbook

Connect to MS SQL Server

In [7]:
import pyodbc

# Connection parameters
server = 'DESKTOP-89C0DB2'
database = 'master'  # Use the master database to create a new database
username = ''  # Update with your username
password = ''  # Update with your password
driver = '{SQL Server}'

# Establishing the connection to the master database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Create the ECommerceDB database
cursor.execute("CREATE DATABASE ECommerceDB")
cursor.commit()
print("Created ECommerceDB Successfully!")
# # Close the connection
cursor.close()
conn.close()

Created ECommerceDB Successfully!


In [8]:
import pyodbc

# Connection parameters
server = 'DESKTOP-89C0DB2'
database = 'ECommerceDB'  # Connect to the ECommerceDB database
username = ''  # Update with your username
password = ''  # Update with your password
driver = '{SQL Server}'

# Establishing the connection to the ECommerceDB database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Create the Categories Table first
cursor.execute('''
    CREATE TABLE Categories (
        CategoryID INT PRIMARY KEY,
        CategoryName NVARCHAR(100)
    )
''')

# Create the Customers Table
cursor.execute('''
    CREATE TABLE Customers (
        CustomerID INT PRIMARY KEY,
        FirstName NVARCHAR(50),
        LastName NVARCHAR(50),
        Email NVARCHAR(100),
        SignUpDate DATE
    )
''')

# Create the Products Table
cursor.execute('''
    CREATE TABLE Products (
        ProductID INT PRIMARY KEY,
        ProductName NVARCHAR(100),
        Price FLOAT,
        StockQuantity INT,
        CategoryID INT,
        FOREIGN KEY(CategoryID) REFERENCES Categories(CategoryID)
    )
''')

# Create the Regions Table
cursor.execute('''
    CREATE TABLE Regions (
        RegionID INT PRIMARY KEY,
        RegionName NVARCHAR(100)
    )
''')

# Create the Orders Table
cursor.execute('''
    CREATE TABLE Orders (
        OrderID INT PRIMARY KEY,
        CustomerID INT,
        OrderDate DATE,
        TotalAmount FLOAT,
        RegionID INT,
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID),
        FOREIGN KEY(RegionID) REFERENCES Regions(RegionID)
    )
''')

# Create the OrderDetails Table
cursor.execute('''
    CREATE TABLE OrderDetails (
        OrderDetailID INT PRIMARY KEY,
        OrderID INT,
        ProductID INT,
        Quantity INT,
        UnitPrice FLOAT,
        FOREIGN KEY(OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID)
    )
''')

# Create the Returns Table
cursor.execute('''
    CREATE TABLE Returns (
        ReturnID INT PRIMARY KEY,
        OrderID INT,
        ProductID INT,
        ReturnDate DATE,
        QuantityReturned INT,
        RefundAmount FLOAT,
        FOREIGN KEY(OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID)
    )
''')

# Create the Reviews Table
cursor.execute('''
    CREATE TABLE Reviews (
        ReviewID INT PRIMARY KEY,
        ProductID INT,
        CustomerID INT,
        Rating INT,
        ReviewText NVARCHAR(MAX),
        ReviewDate DATE,
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID),
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    )
''')

# Create the ShippingAddresses Table
cursor.execute('''
    CREATE TABLE ShippingAddresses (
        AddressID INT PRIMARY KEY,
        CustomerID INT,
        Address NVARCHAR(MAX),
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    )
''')

# Commit the changes
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("All tables created successfully in ECommerceDB.")


All tables created successfully in ECommerceDB.


# Insert Dummy Data

In [9]:
# !pip install faker

In [10]:
import pyodbc
import random
from faker import Faker

# Create a Faker instance
fake = Faker()

# Connection parameters
server = 'DESKTOP-89C0DB2'
database = 'ECommerceDB'
username = ''  # Update with your username
password = ''  # Update with your password
driver = '{SQL Server}'

# Establishing the connection to the ECommerceDB database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

n = 1000  # number of dummy records to insert for each table

# Inserting dummy data into Categories
for i in range(1, n + 1):
    cursor.execute("INSERT INTO Categories (CategoryID, CategoryName) VALUES (?, ?)",
                   i, fake.word().capitalize())

# Inserting dummy data into Customers
for i in range(1, n + 1):
    signup_date = fake.date_between(start_date='-5y', end_date='today')
    formatted_signup_date = signup_date.strftime('%Y-%m-%d')  # Convert to string
    cursor.execute("INSERT INTO Customers (CustomerID, FirstName, LastName, Email, SignUpDate) VALUES (?, ?, ?, ?, ?)",
                   i, fake.first_name(), fake.last_name(), fake.email(), formatted_signup_date)

# Inserting dummy data into Products
for i in range(1, n + 1):
    cursor.execute("INSERT INTO Products (ProductID, ProductName, Price, StockQuantity, CategoryID) VALUES (?, ?, ?, ?, ?)",
                   i, fake.word().capitalize(), random.uniform(10.0, 500.0), random.randint(0, 100), random.randint(1, n))

# Inserting dummy data into Regions
for i in range(1, n + 1):
    cursor.execute("INSERT INTO Regions (RegionID, RegionName) VALUES (?, ?)",
                   i, fake.state())

# Inserting dummy data into Orders
for i in range(1, n + 1):
    order_date = fake.date_between(start_date='-5y', end_date='today')
    formatted_order_date = order_date.strftime('%Y-%m-%d')  # Convert to string
    cursor.execute("INSERT INTO Orders (OrderID, CustomerID, OrderDate, TotalAmount, RegionID) VALUES (?, ?, ?, ?, ?)",
                   i, random.randint(1, n), formatted_order_date, random.uniform(100.0, 1000.0), random.randint(1, n))

# Inserting dummy data into OrderDetails
for i in range(1, n + 1):
    cursor.execute("INSERT INTO OrderDetails (OrderDetailID, OrderID, ProductID, Quantity, UnitPrice) VALUES (?, ?, ?, ?, ?)",
                   i, random.randint(1, n), random.randint(1, n), random.randint(1, 10), random.uniform(10.0, 500.0))

# Inserting dummy data into Returns
for i in range(1, n + 1):
    return_date = fake.date_between(start_date='-5y', end_date='today')
    formatted_return_date = return_date.strftime('%Y-%m-%d')  # Convert to string
    cursor.execute("INSERT INTO Returns (ReturnID, OrderID, ProductID, ReturnDate, QuantityReturned, RefundAmount) VALUES (?, ?, ?, ?, ?, ?)",
                   i, random.randint(1, n), random.randint(1, n), formatted_return_date, random.randint(1, 5), random.uniform(10.0, 500.0))

# Inserting dummy data into Reviews
for i in range(1, n + 1):
    review_date = fake.date_between(start_date='-5y', end_date='today')
    formatted_review_date = review_date.strftime('%Y-%m-%d')  # Convert to string
    cursor.execute("INSERT INTO Reviews (ReviewID, ProductID, CustomerID, Rating, ReviewText, ReviewDate) VALUES (?, ?, ?, ?, ?, ?)",
                   i, random.randint(1, n), random.randint(1, n), random.randint(1, 5), fake.text(max_nb_chars=200), formatted_review_date)

# Inserting dummy data into ShippingAddresses
for i in range(1, n + 1):
    cursor.execute("INSERT INTO ShippingAddresses (AddressID, CustomerID, Address) VALUES (?, ?, ?)",
                   i, random.randint(1, n), fake.address())

# Commit the changes
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Dummy data inserted successfully into all tables.")


Dummy data inserted successfully into all tables.


# View Data From the DB

In [11]:
import pyodbc
import pandas as pd

def read_table_as_dataframe(table_name):
    # Connection parameters
    server = 'DESKTOP-89C0DB2'
    database = 'ECommerceDB'
    username = ''  # Update with your username
    password = ''  # Update with your password
    driver = '{SQL Server}'

    # Establishing the connection to the ECommerceDB database
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)

    # Query to read the table
    query = f"SELECT * FROM {table_name}"
    
    # Reading the table into a Pandas DataFrame
    df = pd.read_sql(query, conn)

    # Close the connection
    conn.close()

    return df

# Example usage:
table_name = 'Customers'  # Replace with the name of the table you want to display
df = read_table_as_dataframe(table_name)
print(df)


     CustomerID FirstName  LastName                          Email  SignUpDate
0             1   Yolanda     Yoder           steven23@example.net  2021-05-25
1             2       Joe     Clark  simmonsalexandria@example.org  2019-09-17
2             3   Kristin     Doyle      paynekimberly@example.com  2022-02-24
3             4     Bryan    Rogers     jacksonshelley@example.com  2023-03-24
4             5    Joseph    Wilcox             wirwin@example.org  2023-07-14
..          ...       ...       ...                            ...         ...
995         996      Brad   Fleming        drichardson@example.net  2020-05-06
996         997   Shannon  Williams       torresbianca@example.org  2021-02-10
997         998       Amy    Thomas         mossadrian@example.com  2019-12-20
998         999    Samuel    Miller          olsonlisa@example.org  2023-02-19
999        1000    Edward    Watson         jennifer86@example.org  2021-07-31

[1000 rows x 5 columns]
